In [1]:
import os
os.chdir('..')

from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "mean"
fs_method, fs_ratio = "rfcls", 50/142
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

In [2]:
from utilities import GNNClassifier

model_fn = functools.partial(
    GNNClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GNNClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9111 | AUPR: 0.6533 | Acc: 95.47%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9154 | AUPR: 0.6649 | Acc: 95.37%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9193 | AUPR: 0.6746 | Acc: 95.55%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9109 | AUPR: 0.6506 | Acc: 95.43%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9165 | AUPR: 0.6662 | Acc: 95.38%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9197 | AUPR: 0.6761 | Acc: 95.49%
best model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9197 | AUPR: 0.6761 | Acc: 95.49%
Internal test: AUC: 0.9270 | AUPR: 0.7032 | Acc: 95.33%
External test: AUC: 0.9159 | AUPR: 0.6614 | Acc: 95.08%


In [3]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier
model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9137 | AUPR: 0.6588 | Acc: 95.50%
model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9177 | AUPR: 0.6688 | Acc: 95.51%
model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9202 | AUPR: 0.6785 | Acc: 95.58%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9104 | AUPR: 0.6501 | Acc: 95.40%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9157 | AUPR: 0.6644 | Acc: 95.40%
model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9201 | AUPR: 0.6762 | Acc: 95.55%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9202 | AUPR: 0.6785 | Acc: 95.58%
Internal test: AUC: 0.9280 | AUPR: 0.7057 | Acc: 95.40%
External test: AUC: 0.9169 | AUPR: 0.6677 | Acc: 95.22%


In [4]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression
model(C=1.0)
Valid:         AUC: 0.9118 | AUPR: 0.6482 | Acc: 95.35%
model(C=10.0)
Valid:         AUC: 0.9115 | AUPR: 0.6476 | Acc: 95.29%
model(C=100.0)
Valid:         AUC: 0.9113 | AUPR: 0.6473 | Acc: 95.29%
best model(C=1.0)
Valid:         AUC: 0.9118 | AUPR: 0.6482 | Acc: 95.35%
Internal test: AUC: 0.9199 | AUPR: 0.6814 | Acc: 95.35%
External test: AUC: 0.9059 | AUPR: 0.6233 | Acc: 94.85%


In [5]:
import warnings
from pytorch_tabnet.tab_model import TabNetClassifier
warnings.filterwarnings("ignore", category=UserWarning)

model_fn = functools.partial(
    TabNetClassifier, seed=SEED, 
    n_d=4, n_a=4, momentum=0.05, 
    n_steps=3, gamma=1.75, cat_emb_dim=1,
    n_independent=2, n_shared=2,
    verbose=0)
param_grid_list = [{
    'optimizer_params': [dict(lr=lr) for lr in [5e-2]],
    'gamma': [1.5, 1.75, 2.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)
warnings.resetwarnings()

TabNetClassifier
model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9045 | AUPR: 0.6279 | Acc: 95.13%
model(optimizer_params={'lr': 0.05}, gamma=1.75)
Valid:         AUC: 0.9006 | AUPR: 0.6325 | Acc: 95.26%
model(optimizer_params={'lr': 0.05}, gamma=2.0)
Valid:         AUC: 0.9012 | AUPR: 0.6160 | Acc: 94.98%
best model(optimizer_params={'lr': 0.05}, gamma=1.5)
Valid:         AUC: 0.9045 | AUPR: 0.6279 | Acc: 95.13%
Internal test: AUC: 0.9079 | AUPR: 0.6718 | Acc: 95.60%
External test: AUC: 0.8959 | AUPR: 0.6184 | Acc: 94.98%


In [6]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9147 | AUPR: 0.6582 | Acc: 89.22%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9149 | AUPR: 0.6595 | Acc: 89.38%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9153 | AUPR: 0.6601 | Acc: 89.38%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9150 | AUPR: 0.6607 | Acc: 89.41%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9154 | AUPR: 0.6610 | Acc: 89.59%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9156 | AUPR: 0.6627 | Acc: 89.62%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9150 | AUPR: 0.6607 | Acc: 89.41%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9154 | AUPR: 0.6610 | Acc: 89.59%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9156 | AUPR: 0.6627 | Acc: 89.62%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9156 | AUPR: 0.6627 | Ac

In [7]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9120 | AUPR: 0.6112 | Acc: 95.01%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9119 | AUPR: 0.6023 | Acc: 94.88%
model(kernel=linear, C=1)
Valid:         AUC: 0.9105 | AUPR: 0.6016 | Acc: 94.87%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9120 | AUPR: 0.6112 | Acc: 95.01%
Internal test: AUC: 0.9193 | AUPR: 0.6428 | Acc: 94.62%
External test: AUC: 0.9092 | AUPR: 0.5886 | Acc: 94.47%


In [8]:
from sklearn.ensemble import GradientBoostingClassifier

model_fn = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9174 | AUPR: 0.6551 | Acc: 95.37%
model(criterion=squared_error)
Valid:         AUC: 0.9175 | AUPR: 0.6548 | Acc: 95.34%
best model(criterion=squared_error)
Valid:         AUC: 0.9175 | AUPR: 0.6548 | Acc: 95.34%
Internal test: AUC: 0.9295 | AUPR: 0.6940 | Acc: 95.26%
External test: AUC: 0.9188 | AUPR: 0.6635 | Acc: 95.20%


In [9]:
from sklearn.ensemble import AdaBoostClassifier
model_fn = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9121 | AUPR: 0.6320 | Acc: 95.05%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9132 | AUPR: 0.6331 | Acc: 95.11%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9128 | AUPR: 0.6291 | Acc: 95.07%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9132 | AUPR: 0.6331 | Acc: 95.11%
Internal test: AUC: 0.9198 | AUPR: 0.6735 | Acc: 95.21%
External test: AUC: 0.9083 | AUPR: 0.6171 | Acc: 94.72%


In [10]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7752 | AUPR: 0.4546 | Acc: 94.93%
model(n_neighbors=5)
Valid:         AUC: 0.7998 | AUPR: 0.5004 | Acc: 95.25%
model(n_neighbors=7)
Valid:         AUC: 0.8138 | AUPR: 0.5288 | Acc: 95.29%
best model(n_neighbors=7)
Valid:         AUC: 0.8138 | AUPR: 0.5288 | Acc: 95.29%
Internal test: AUC: 0.8322 | AUPR: 0.5575 | Acc: 95.06%
External test: AUC: 0.8229 | AUPR: 0.5396 | Acc: 95.11%


In [11]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8615 | AUPR: 0.5543 | Acc: 89.07%
model(criterion=log_loss)
Valid:         AUC: 0.8619 | AUPR: 0.5725 | Acc: 89.37%
model(criterion=entropy)
Valid:         AUC: 0.8619 | AUPR: 0.5725 | Acc: 89.37%
best model(criterion=log_loss)
Valid:         AUC: 0.8619 | AUPR: 0.5725 | Acc: 89.37%
Internal test: AUC: 0.8773 | AUPR: 0.6116 | Acc: 90.51%
External test: AUC: 0.8665 | AUPR: 0.5867 | Acc: 89.94%


In [12]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8819 | AUPR: 0.4603 | Acc: 86.47%
best model()
Valid:         AUC: 0.8819 | AUPR: 0.4603 | Acc: 86.47%
Internal test: AUC: 0.8941 | AUPR: 0.5157 | Acc: 87.20%
External test: AUC: 0.8811 | AUPR: 0.4672 | Acc: 85.71%


In [13]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)